**CS109A Final Project**<br/>
**Group 12**<br/>
**EDA Page**<br/>

Import libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.api import OLS
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import seaborn as sns; sns.set(color_codes=True)
sns.set(style="whitegrid")


/Users/anantpai/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Load in data
df = pd.read_csv('total_info.csv', encoding = 'latin-1')

In [3]:
# Clean Variables
del df['Unnamed: 0']
df['artist_popularity'] = pd.to_numeric(df['artist_popularity'], errors = 'coerce')
df['artist_followers'] = pd.to_numeric(df['artist_followers'], errors = 'coerce')

In [4]:
# Create pop indicator variable array
pop_ind = []
rap_ind = []
rock_ind = []
hip_hop_ind = []
country_ind = []
metal_ind = []
indie_folk_ind = []
rnb_ind = []
edm_ind = []
classical_ind = []
jazz_ind = []

for idx, row in enumerate(df['artist_genres'].values):
    if 'pop' in row:
        pop_ind.append(1)
    else:
        pop_ind.append(0)
        
    if 'rap' in row or 'hip' in row:
        rap_ind.append(1)
    else:
        rap_ind.append(0)
        
    if 'rock' in row:
        rock_ind.append(1)
    else:
        rock_ind.append(0)
    
    if 'hip' in row or 'hop' in row:
        hip_hop_ind.append(1)
    else:
        hip_hop_ind.append(0)
    
    if 'country' in row:
        country_ind.append(1)
    else:
        country_ind.append(0)
        
    if 'metal' in row:
        metal_ind.append(1)
    else:
        metal_ind.append(0)
        
    if 'indie' in row or 'folk' in row:
        indie_folk_ind.append(1)
    else:
        indie_folk_ind.append(0)
        
    if 'r&b' in row:
        rnb_ind.append(1)
    else:
        rnb_ind.append(0)
        
    if 'edm' in row:
        edm_ind.append(1)
    else:
        edm_ind.append(0)
    
    if 'classical' in row:
        classical_ind.append(1)
    else:
        classical_ind.append(0)
    
    if 'jazz' in row:
        jazz_ind.append(1)
    else:
        jazz_ind.append(0)
    
        
# Append array to dataframe
df['pop_ind'] = pop_ind
df['rap_ind'] = rap_ind
df['rock_ind'] = rock_ind
df['hip_hop_ind'] = hip_hop_ind
df['country_ind'] = country_ind
df['metal_ind'] = metal_ind
df['indie_folk_ind'] = indie_folk_ind
df['rnb_ind'] = rnb_ind
df['edm_ind'] = edm_ind
df['classical_ind'] = classical_ind
df['jazz_ind'] = jazz_ind

In [5]:
# Artist Popularity 75% is a 75, use for indicator 
df['artist_popularity'].describe()
df['popular_artist_ind'] = (df['artist_popularity'] >= 75)*1
df.head()

,playlist_id,playlist_name,followers,song_name,number_of_artists,artist_name,artist_id,popularity,track_number,explicit,...,rock_ind,hip_hop_ind,country_ind,metal_ind,indie_folk_ind,rnb_ind,edm_ind,classical_ind,jazz_ind,popular_artist_ind
0,37i9dQZF1DXcBWIGoYBM5M,Today's Top Hits,18123888.0,Wolves,2,Selena Gomez,0C8ZW7ezQVs4URX5aX7Kqx,88,1,0,...,0,0,0,0,0,0,0,0,0,1
1,37i9dQZF1DWXDAhqlN7e6W,This Is: Max Martin,15685.0,Hands To Myself,1,Selena Gomez,0C8ZW7ezQVs4URX5aX7Kqx,69,3,0,...,0,0,0,0,0,0,0,0,0,1
2,37i9dQZF1DX7Q7o98uPeg1,Funkst,397098.0,Good For You - KASBO Remix,3,Selena Gomez,0C8ZW7ezQVs4URX5aX7Kqx,35,3,0,...,0,0,0,0,0,0,0,0,0,1
3,37i9dQZF1DX3LyU0mhfqgP,Out Now,439685.0,Wolves,2,Selena Gomez,0C8ZW7ezQVs4URX5aX7Kqx,88,1,0,...,0,0,0,0,0,0,0,0,0,1
4,37i9dQZF1DX35oM5SPECmN,HIIT Workout,712771.0,Come & Get It,1,Selena Gomez,0C8ZW7ezQVs4URX5aX7Kqx,56,1,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
international = []
genre_or_not = []

for idx, row in enumerate(df['available_markets'].values):
    if len(str(row)) > 3:
        international.append(1)
    else:
        international.append(0)
        
df['international_or_not'] = international
df['genre_or_not'] = genre_or_not

In [7]:
columns = ['playlist','name','followers','num_songs','av_song_pop','pct_explicit','avg_dur','av_artist_followers', 'pop_pct', 'rap_pct', 'rock_pct', 'hip_hop_pct', 'country_pct','metal_pct','indie_folk_pct','rnb_pct','edm_pct','classical_pct','jazz_pct','popular_artist_pct', 'top_3_artist_av_pop', 'top_artist_pop', 'top_3_song_avg_pop', 'top_song_pop', 'top_3_artist_followers', 'top_artist_followers', 'percent_international', 'diff_song_pop', 'diff_artist_pop']
len(columns)

29

In [9]:
# Sum Up the 100 tracks of info (or less) for each playlist
unique_plists = df['playlist_id'].unique()
columns = ['playlist','name','followers','num_songs','av_song_pop','pct_explicit','avg_dur','av_artist_followers', 'pop_pct', 'rap_pct', 'rock_pct', 'hip_hop_pct', 'country_pct','metal_pct','indie_folk_pct','rnb_pct','edm_pct','classical_pct','jazz_pct','popular_artist_pct', 'top_3_artist_av_pop', 'top_artist_pop', 'top_3_song_avg_pop', 'top_song_pop', 'top_3_artist_followers', 'top_artist_followers', 'percent_international', 'diff_song_pop', 'diff_artist_pop']
eda_frame = pd.DataFrame(index=range(0,len(unique_plists)), columns=columns)
for idx,plist in enumerate(unique_plists):
    eda_frame.loc[idx]['playlist'] = plist
    eda_frame.loc[idx]['name'] = df.loc[df['playlist_id'] == plist]['playlist_name'].iloc[0]
    eda_frame.loc[idx]['followers'] = np.mean(df.loc[df['playlist_id'] == plist]['followers'])
    eda_frame.loc[idx]['num_songs'] = len(df.loc[df['playlist_id'] == plist])
    eda_frame.loc[idx]['av_song_pop'] = np.mean(df.loc[df['playlist_id'] == plist]['popularity'])
    eda_frame.loc[idx]['pct_explicit'] = np.mean(df.loc[df['playlist_id'] == plist]['explicit'])
    eda_frame.loc[idx]['avg_dur'] = np.mean(df.loc[df['playlist_id'] == plist]['duration_ms'])
    eda_frame.loc[idx]['av_artist_followers'] = np.mean(df.loc[df['playlist_id'] == plist]['artist_followers'])
    eda_frame.loc[idx]['pop_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['pop_ind'])
    eda_frame.loc[idx]['rap_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['rap_ind'])
    eda_frame.loc[idx]['rock_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['rock_ind'])
    eda_frame.loc[idx]['hip_hop_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['hip_hop_ind'])
    eda_frame.loc[idx]['country_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['country_ind'])
    eda_frame.loc[idx]['metal_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['metal_ind'])
    eda_frame.loc[idx]['indie_folk_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['indie_folk_ind'])
    eda_frame.loc[idx]['rnb_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['rnb_ind'])
    eda_frame.loc[idx]['edm_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['edm_ind'])
    eda_frame.loc[idx]['classical_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['classical_ind'])
    eda_frame.loc[idx]['jazz_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['jazz_ind'])
    eda_frame.loc[idx]['popular_artist_pct'] = np.mean(df.loc[df['playlist_id'] == plist]['popular_artist_ind'])
    artist_pop_arr = df.loc[df['playlist_id'] == plist]['artist_popularity'].values
    top_artist_pop_arr = np.partition(-artist_pop_arr, 3)
    eda_frame.loc[idx]['top_3_artist_av_pop'] = np.mean(-top_artist_pop_arr[:3])
    eda_frame.loc[idx]['top_artist_pop'] = max(artist_pop_arr)
    song_pop_arr = df.loc[df['playlist_id'] == plist]['popularity'].values
    top_song_pop_arr = np.partition(-song_pop_arr, 3)
    eda_frame.loc[idx]['top_3_song_avg_pop'] = np.mean(-top_song_pop_arr[:3])
    eda_frame.loc[idx]['top_song_pop'] = max(song_pop_arr)
    followers_arr = df.loc[df['playlist_id'] == plist]['artist_followers'].values
    top_followers_arr = np.partition(-followers_arr, 3)
    eda_frame.loc[idx]['top_3_artist_followers'] = np.mean(-top_followers_arr[:3])
    eda_frame.loc[idx]['top_artist_followers'] = max(followers_arr)
    eda_frame.loc[idx]['percent_international'] = np.mean(df.loc[df['playlist_id'] == plist]['international_or_not'])
    eda_frame.loc[idx]['diff_song_pop'] = max(df.loc[df['playlist_id'] == plist]['popularity']) - min(df.loc[df['playlist_id'] == plist]['popularity'])
    eda_frame.loc[idx]['diff_artist_pop'] = max(df.loc[df['playlist_id'] == plist]['artist_popularity']) - min(df.loc[df['playlist_id'] == plist]['artist_popularity'])

In [14]:
len(eda_frame.columns)

32

In [12]:
top_indicator = []
this_is_indicator = []
num_indicator = []

for name in eda_frame['name']:     
    if 'Top' in name or 'Best' in name or 'Hit' in name or 'Viral' in name or 'Hot' in name:
        top_indicator.append(1)
    else:
        top_indicator.append(0)
    if "This Is:" in name:
        this_is_indicator.append(1)
    else:
        this_is_indicator.append(0)
    if any(c.isdigit() for c in name):
        num_indicator.append(1)
    else:
        num_indicator.append(0)
        
eda_frame['top_or_not'] = top_indicator
eda_frame['this_or_not'] = this_is_indicator
eda_frame['number_or_not'] = num_indicator

In [13]:
len(eda_frame.columns)

32

In [15]:
# Dummies for most popular artists
artist_counts = df.groupby('artist_name', as_index=False)['artist_popularity'].mean().sort_values('artist_popularity')
top_artists = artist_counts[artist_counts['artist_popularity']>=93]
artist_counts = artist_counts.fillna(0)

top_artists['artist_name'] = list(top_artists['artist_name'])

top_artist_dummy = pd.get_dummies(top_artists['artist_name'])

df_new = pd.concat([artist_counts, top_artist_dummy], axis=1)
df_new = df_new.drop('artist_popularity', axis =1)

df_artists = pd.merge(df, df_new, on='artist_name')
df_artists = df_artists.fillna(0)

In [16]:
artists = list(top_artists['artist_name'])
columns = artists[:]
columns.insert(0,'name')
playlist_artist = pd.DataFrame(index=range(0,len(unique_plists)), columns=columns)

In [17]:
for i, val in enumerate(unique_plists):
    playlist_artist['name'].values[i] = df_artists[df_artists['playlist_id'] == val]['playlist_name'].values[0]
    for col in artists:
        playlist_artist.set_value(i, col, max(df_artists.loc[df['playlist_id'] == val][col])) 

In [18]:
eda_frame = eda_frame.merge(playlist_artist, on = 'name')

In [19]:
len(eda_frame.columns)

72

In [21]:
genre_or_not = []

for idx, row in enumerate(df['available_markets'].values):
    if len(str(row)) > 3:
        genre_or_not.append(1)
    else:
        genre_or_not.append(0)
        
df['genre_or_not'] = genre_or_not

In [22]:
columns = ['playlist','pct_w_genre']
extra_frame = pd.DataFrame(index=range(0,len(unique_plists)), columns=columns)
for idx,plist in enumerate(unique_plists):
    extra_frame.loc[idx]['playlist'] = plist
    extra_frame.loc[idx]['pct_w_genre'] = np.mean(df.loc[df['playlist_id'] == plist]['genre_or_not'])


In [24]:
eda_frame = eda_frame.merge(extra_frame, on = 'playlist')

In [25]:
len(eda_frame.columns)

73

In [26]:
len(eda_frame)

1594

73 predictors and 1594 observations! Great :)

Now, we've put all of our predictors together. Let's go ahead and create interaction terms for the variables we may think are important

In [63]:
master_frame = eda_frame

In [64]:
# Create Interaction Terms
artist_list = ['Demi Lovato', 'Lil Wayne','BeyoncÌÄå©', 'Selena Gomez', 'SZA', 'Marshmello', 'Coldplay','Kanye West', 'Kygo', 'Dua Lipa', 'XXXTENTACION', 'Sia','Camila Cabello', 'Migos', 'Gucci Mane', 'Imagine Dragons', 'Rihanna','Travis Scott', 'Maroon 5', 'Taylor Swift', 'The Weeknd','Ty Dolla $ign', 'Nicki Minaj', 'Daddy Yankee', 'Young Thug','Lil Uzi Vert', 'Kendrick Lamar', 'Eminem', 'Chris Brown', 'Future','Justin Bieber', 'J Balvin', 'Sam Smith', '21 Savage', 'Ozuna','Bad Bunny', 'Khalid', 'Ed Sheeran', 'Drake', 'Post Malone']
genres = ['pop_pct', 'rap_pct', 'rock_pct', 'hip_hop_pct','country_pct', 'metal_pct', 'indie_folk_pct', 'rnb_pct', 'edm_pct','classical_pct', 'jazz_pct']

for artist in artist_list:
    for genre in genres:
        master_frame[artist+'*'+ genre] = master_frame[artist] * master_frame[genre]

In [65]:
len(master_frame.columns)

513

Let's go ahead and split up into train and test data. This way we can normalize/ standardize properly.

In [66]:
np.log(513.0)

6.2402758451707694

In [67]:
# First, let's take the log to fix the skew of certain variables
skewed_vars = ['followers','av_artist_followers', 'top_3_artist_followers', 'top_artist_followers']
for var in skewed_vars:
    master_frame[var] = np.log(master_frame[var].astype('float'))

Since we now have values that are negative infinity (since there were 0s before logging, let's go ahead and replace them with 0s because the difference between an original 0 and 1 in the data is basically trivial. 

In [68]:
master_frame = master_frame.replace([np.inf, -np.inf], 0)

In [69]:
# Split into Train and Test
np.random.seed(9001)
msk = np.random.rand(len(master_frame)) < 0.70

data_train = master_frame[msk]
data_test = master_frame[~msk]

In [70]:
len(data_train)

1106

Let's standardize our predictors.

In [71]:
# Standardize
continuous_vars = ['followers','num_songs','av_song_pop','pct_explicit','avg_dur','av_artist_followers', 'pop_pct', 'rap_pct', 'rock_pct', 'hip_hop_pct', 'country_pct','metal_pct','indie_folk_pct','rnb_pct','edm_pct','classical_pct','jazz_pct','popular_artist_pct', 'top_3_artist_av_pop', 'top_artist_pop', 'top_3_song_avg_pop', 'top_song_pop', 'top_3_artist_followers', 'top_artist_followers', 'percent_international', 'diff_song_pop', 'diff_artist_pop']
for var in continuous_vars: 
    data_test[var] = (data_test[var] - data_train[var].mean())/data_train[var].std()
    data_train[var] = (data_train[var] - data_train[var].mean())/data_train[var].std()

In [72]:
data_train[var].mean()

4.2022970395805613e-16

In [73]:
data_train.to_csv('train_data.csv')
data_test.to_csv('test_data.csv')